In [1]:
from nplm import PartialLabelModel
import numpy as np
import torch
import joblib

%load_ext autoreload
%autoreload 2

In [2]:
typ = 'A'
split = 0

############################
# 1. Load Detectors Votes
votes = np.load('votes/lad_unseen_votes_{:s}{:d}.npy'.format(typ, split))


############################
# 2. Load Class-Attribute CFG (Specifying attribute as partial label) as fid2clusters
fid2clusters = np.load('data/lad/fid2clusters_{:s}{:d}.npy'.format(typ, split), allow_pickle=True).item()

In [3]:
lm_cfg = np.load('cfgs/LAD/lad_cfgs.npy', allow_pickle=True).item()['{:s}{:d}'.format(typ, split)]

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
labelmodel = PartialLabelModel(num_classes=10, # 6 for hair
                            fid2clusters=fid2clusters,
                            opt_cfg=lm_cfg,
                            device=device, verbose=True)

labelmodel.optimize(votes)

epoch   6:   0%|          | 0/1 [00:00<?, ?it/s]

Setup:  0.08187365531921387


epoch  112: 100%|██████████| 1/1 [00:00<00:00, 27.71it/s, Epoch Loss: =19.3]


epoch  120: 100%|██████████| 1/1 [00:00<00:00, 27.74it/s, Epoch Loss: =19.3]


In [4]:
label_estim = labelmodel.weak_label(votes)

Setup:  0.06451964378356934
Parallel Estimation:  0.009848356246948242


In [8]:
import pickle

def ladpipeline(softlabels, split, typ):

        data_pac = pickle.load(
            open('data/LAD/splits/split_{:d}/{:s}/r50_features/unseen_all.pkl'.format(split, typ),
                 'rb'))
        feat_dim = 2048
        features, labels = zip(*data_pac)

        meta_split_info = np.load('data/lad/meta_split_info_v2.npy', allow_pickle=True).item()
        unseen_classes = meta_split_info['splits'][split][typ]['unseen']

        class_name_map = {old_cid: new_cid for new_cid, old_cid in enumerate(unseen_classes)}

        translator = {old:new for new, old in enumerate(np.unique(labels))}
        translated_labels = [class_name_map[int(old)] for old in labels]
        soft_labels = softlabels['{:s}-{:d}'.format(typ, split)]
        train_data = list(zip(features, soft_labels))
        test_data = list(zip(features, translated_labels))

        train = DataLoader(train_data, shuffle=True, batch_size=64)
        valtest = DataLoader(test_data, shuffle=True, batch_size=64)

        model_const = ResNetFeaturesLC
        task = EndTask('LAD-{:s}-{:d}'.format(typ, split), len(np.unique(labels)), model_const,
                       feat_dim=feat_dim)
        task.train(train, valtest, valtest, save_model=False, double=False)

In [ ]:
ladpipeline(label_estim, split, typ)